In [1]:
from vmad import autooperator
from utils import finite_difference
import numpy as np

\begin{equation}
\begin{split}
    g = \dfrac{f(x+\epsilon) - f(x)}{\epsilon}\\
\end{split}
\end{equation}

In [2]:
@autooperator('a->b')
def test(a, func, epsilon, args=(), mode='forward'):
    return finite_difference(a, func, epsilon, args, mode)

def test_func(a, j,k):
    return a**3+j*a**2+k*a

In [6]:
epsilon = 1e-5
j,k = 1,2
init = dict(a=1)
v = dict(_b=1)

apl_f, vjp_f = test.build(func=test_func, epsilon=.001, args=(j,k), mode='forward').compute_with_vjp(init=init, v=v)
apl_b, vjp_b = test.build(func=test_func, epsilon=.001, args=(j,k), mode='backward').compute_with_vjp(init=init, v=v)
apl_c, vjp_c = test.build(func=test_func, epsilon=.001, args=(j,k), mode='central').compute_with_vjp(init=init, v=v)

assert 


print('apl: {}, vjp: {}'.format(apl[0], vjp[0]))

apl: 6.996001000000085, vjp: 6.996001000000085


# Cosmology test

In [194]:
from nbodykit.cosmology import Planck15, LinearPower
from fastpm import background

In [193]:
cosmo = Planck15

In [200]:
testing = background.MatterDominated(cosmo.Omega0_m)

In [201]:
model = test.build(func=, epsilon=.01,args=(1,2))

SyntaxError: invalid syntax (<ipython-input-201-056d7fd2c7df>, line 1)

In [186]:
model.compute_with_vjp(init=dict(a = cosmo.Omega0_m), v=dict(_b=1))

TypeError: get_pklin() takes exactly 2 positional arguments (3 given)